In [1]:
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
#import holidays
from prophet import Prophet
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [3]:
data_scientist = "dlorino"

In [21]:
campaign_id = "WOW20000860_1"
train_period = client.query(
f"""
DECLARE media_start_date_global_var DATE; 
DECLARE media_end_date_global_var DATE; 
SET media_start_date_global_var = (SELECT DISTINCT media_start_date FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_campaign_asset_info WHERE booking_and_asset_number = '{campaign_id}');
SET media_end_date_global_var = (SELECT DISTINCT media_end_date FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_campaign_asset_info WHERE booking_and_asset_number = '{campaign_id}');
SELECT 
    BusinessDate AS ds, 
    SUM(TotalAmountIncludingGST) AS y
--FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_campaign_period_transactions
FROM `gcp-wow-wiq-ca-prod.wiqIN_DataAssets.CustomerBaseTransaction_v` transactions
INNER JOIN cart-dai-sandbox-nonprod-c3e7.{data_scientist}.unique_skus_2 skus 
    ON TRIM(skus.sku) = transactions.Article 
WHERE BusinessDate >= DATE_ADD(DATE_ADD(media_start_date_global_var, INTERVAL -4 WEEK), INTERVAL -2 YEAR)
AND BusinessDate <= DATE_ADD(media_start_date_global_var, INTERVAL -4 WEEK)
GROUP BY 1
ORDER BY 1
"""
).result()
train_period_df = train_period.to_dataframe()
chime.success()
train_period_df 

,ds,y
0,2021-04-15,19848.040000000
1,2021-04-16,20674.530000000
2,2021-04-17,23285.770000000
3,2021-04-18,26605.400000000
4,2021-04-19,22785.930000000
...,...,...
695,2023-03-11,30523.950000000
696,2023-03-12,32687.500000000
697,2023-03-13,30099.130000000
698,2023-03-14,27809.400000000


In [23]:
campaign_id = "WOW20000860_1"
forecast_window = client.query(
f"""
DECLARE media_start_date_global_var DATE; 
DECLARE media_end_date_global_var DATE; 
SET media_start_date_global_var = (SELECT DISTINCT media_start_date FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_campaign_asset_info WHERE booking_and_asset_number = '{campaign_id}');
SET media_end_date_global_var = (SELECT DISTINCT media_end_date FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_campaign_asset_info WHERE booking_and_asset_number = '{campaign_id}');
SELECT 
    BusinessDate AS ds, 
    SUM(TotalAmountIncludingGST) AS y
--FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_campaign_period_transactions
FROM `gcp-wow-wiq-ca-prod.wiqIN_DataAssets.CustomerBaseTransaction_v` transactions
INNER JOIN cart-dai-sandbox-nonprod-c3e7.{data_scientist}.unique_skus_2 skus 
    ON TRIM(skus.sku) = transactions.Article 
WHERE BusinessDate > DATE_ADD(media_start_date_global_var, INTERVAL -4 WEEK)
AND BusinessDate < media_start_date_global_var
GROUP BY 1
ORDER BY 1
"""
).result()
forecast_window_df = forecast_window.to_dataframe()
chime.success()
forecast_window_df 

,ds,y
0,2023-03-16,26850.480000000
1,2023-03-17,26273.040000000
2,2023-03-18,31736.620000000
3,2023-03-19,34661.380000000
4,2023-03-20,29513.550000000
5,2023-03-21,25487.840000000
6,2023-03-22,18278.380000000
7,2023-03-23,16865.260000000
8,2023-03-24,16732.760000000
9,2023-03-25,21108.700000000


In [26]:
#print(holidays.country_holidays('AU'))
m = Prophet()
m.add_country_holidays('AU')
m.fit(train_period_df)
m.train_holiday_names

15:05:16 - cmdstanpy - INFO - Chain [1] start processing
15:05:16 - cmdstanpy - INFO - Chain [1] done processing


0                                        Good Friday
1                                      Easter Monday
2                                     New Year's Day
3                                      Australia Day
4                                          ANZAC Day
5                                      Christmas Day
6                                         Boxing Day
7    National Day of Mourning for Queen Elizabeth II
dtype: object

In [27]:
future = m.make_future_dataframe(periods=28)
future

,ds
0,2021-04-15
1,2021-04-16
2,2021-04-17
3,2021-04-18
4,2021-04-19
...,...
723,2023-04-08
724,2023-04-09
725,2023-04-10
726,2023-04-11


In [33]:
forecast = m.predict(future)
forecast['ds'] = pd.to_datetime(forecast['ds'])
forecast_window_df['ds'] = pd.to_datetime(forecast_window_df['ds'])
comparison_df = pd.merge(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], forecast_window_df, on='ds')

In [36]:
# Convert both 'y' and 'yhat' to float
comparison_df['y'] = comparison_df['y'].astype(float)
comparison_df['yhat'] = comparison_df['yhat'].astype(float)

# Calculate residuals (forecast error)
comparison_df['residual'] = comparison_df['yhat'] - comparison_df['y']

# Calculate percentage error based on the real value (observed value)
comparison_df['percentage_error'] = (comparison_df['residual'] / abs(comparison_df['y'])) * 100

# Display the comparison DataFrame
comparison_df[['ds', 'y', 'yhat', 'residual', 'percentage_error']]


,ds,y,yhat,residual,percentage_error
0,2023-03-16,26850.48,24273.317555,-2577.162445,-9.598199
1,2023-03-17,26273.04,23739.821661,-2533.218339,-9.641893
2,2023-03-18,31736.62,26751.531348,-4985.088652,-15.707686
3,2023-03-19,34661.38,28413.708006,-6247.671994,-18.024880
4,2023-03-20,29513.55,26060.595149,-3452.954851,-11.699558
5,2023-03-21,25487.84,23958.509807,-1529.330193,-6.000235
6,2023-03-22,18278.38,24245.855885,5967.475885,32.647729
7,2023-03-23,16865.26,24422.938031,7557.678031,44.812105
8,2023-03-24,16732.76,23889.442137,7156.682137,42.770482
9,2023-03-25,21108.70,26901.151825,5792.451825,27.441064


In [14]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [37]:
check = client.query(
f"""
SELECT * 
FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_baseline_statistics_with_campaign
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,campaign_id,Site,weeks_count,weekly_avg_sales_amount,stddev_sales_amount
0,WOW20000860_1,3800,2,3.675000000,1.237437
1,WOW20000860_1,3433,2,3.360000000,0.000000
2,WOW20000860_1,1393,3,2.290000000,0.441362
3,WOW20000860_1,8882,4,2.502500000,0.545245
4,WOW20000860_1,2755,6,3.256250000,1.833435
...,...,...,...,...,...
1005,WOW20000860_1,1269,12,23.251098901,12.832171
1006,WOW20000860_1,1377,12,14.118571429,8.747844
1007,WOW20000860_1,3343,12,14.337466667,10.053219
1008,WOW20000860_1,3102,12,12.030833333,7.779501


In [40]:
# Match Maker Python Implementation (replaces the C++ version)
def store_matching(df):
    # Helper function for greedy matching
    def greedy_matching(store_pairs):
        matched_pairs = []
        used_test_stores = set()
        used_control_stores = set()

        for _, row in store_pairs.iterrows():
            if row['test_store'] not in used_test_stores and row['control_store'] not in used_control_stores:
                matched_pairs.append(row)
                used_test_stores.add(row['test_store'])
                used_control_stores.add(row['control_store'])

        return pd.DataFrame(matched_pairs)

    # Helper function for global matching
    def global_matching(store_pairs):
        store_pairs_sorted = store_pairs.sort_values('abs_perc_diff')
        return greedy_matching(store_pairs_sorted)

    # Calculate total difference
    def calculate_total_difference(matched_pairs):
        return matched_pairs['abs_perc_diff'].sum()

    # Perform greedy matching
    greedy_result = greedy_matching(df)
    greedy_total_diff = calculate_total_difference(greedy_result)

    # Perform global matching
    global_result = global_matching(df)
    global_total_diff = calculate_total_difference(global_result)

    # Print results
    print(f"Greedy Matching Total Difference: {greedy_total_diff}")
    print(f"Global Matching Total Difference: {global_total_diff}")

    # Return the global matching result as a DataFrame
    return global_result

In [41]:
check = client.query(
f"""

        with step_one AS (
        SELECT baseline.*, campaign.test_or_control 
        FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_baseline_statistics_with_campaign baseline
        LEFT JOIN (SELECT DISTINCT campaign_id, Site, test_or_control FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_campaign_period_transactions) campaign
            ON baseline.campaign_id = campaign.campaign_id 
            AND baseline.Site = campaign.Site
        WHERE baseline.campaign_id = '{campaign_id}'
        AND weeks_count = 12
        ) 
        SELECT 
            test.campaign_id AS study_id, 
            test.Site AS test_store, 
            control.Site AS control_store, 
            ABS(test.weekly_avg_sales_amount / control.weekly_avg_sales_amount - 1) + ABS(test.stddev_sales_amount / control.stddev_sales_amount - 1) AS abs_perc_diff
        FROM step_one test
        LEFT JOIN step_one control 
            ON test.Site <> control.Site 
        WHERE test.test_or_control = "Test" 
        AND control.test_or_control = "Control" 
    """
).result()
historical_performance_df = check.to_dataframe()

chime.success() 

print(f"Creating matched pairs: {campaign_id}")
matched_pairs = store_matching(historical_performance_df)
matched_pairs

Creating matched pairs: WOW20000860_1
Greedy Matching Total Difference: 82.90161209867202
Global Matching Total Difference: 2.14709626117377


,study_id,test_store,control_store,abs_perc_diff
2170,WOW20000860_1,2640,3175,0.000400
67237,WOW20000860_1,1198,2610,0.002398
65007,WOW20000860_1,3395,1470,0.002417
16729,WOW20000860_1,3210,3126,0.002912
60266,WOW20000860_1,3160,1167,0.003415
32212,WOW20000860_1,2670,2951,0.003636
32185,WOW20000860_1,1190,3368,0.003767
36556,WOW20000860_1,3321,1368,0.004450
17916,WOW20000860_1,5654,3053,0.004607
76240,WOW20000860_1,1147,1154,0.004911


In [42]:
check = client.query(
f"""
with step_one AS (
    SELECT 
        cw.campaign_id, 
        cw.Site, 
        baseline.weekly_avg_sales_amount, 
        baseline.stddev_sales_amount, 
        SUM(sales_amount) AS campaign_week_sales 
    FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_campaign_period_transactions cw
    LEFT JOIN cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_baseline_statistics_with_campaign baseline 
        ON baseline.Site = cw.Site
    WHERE baseline.weeks_count = 12
    GROUP BY 1,2,3,4
)

    SELECT 
        campaign_id, 
        Site, 
        weekly_avg_sales_amount,
        stddev_sales_amount,
        campaign_week_sales, 
        (campaign_week_sales - weekly_avg_sales_amount) / weekly_avg_sales_amount AS percentage_uplift_vs_historical_avg
    FROM step_one


"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,campaign_id,Site,weekly_avg_sales_amount,stddev_sales_amount,campaign_week_sales,percentage_uplift_vs_historical_avg
0,WOW20000860_1,1002,27.333578947,16.044549,328.860000000,11.031355302
1,WOW20000860_1,1008,16.697090909,13.654044,131.920000000,6.900777490
2,WOW20000860_1,1030,31.274042553,20.393541,258.590000000,7.268518519
3,WOW20000860_1,1034,18.006307692,10.574011,97.620000000,4.421433515
4,WOW20000860_1,1050,40.929552239,21.535896,326.620000000,6.980053094
...,...,...,...,...,...,...
958,WOW20000860_1,3330,12.591016949,8.337501,54.480000000,3.326894342
959,WOW20000860_1,3382,12.489285714,7.338220,106.170000000,7.500886474
960,WOW20000860_1,4604,12.517500000,8.588851,60.850000000,3.861194328
961,WOW20000860_1,5662,12.159622642,11.884797,75.580000000,5.215653415


In [51]:
# First, merge check_df onto matched_pairs for the test_store
merged_test = pd.merge(
    matched_pairs,
    check_df[['campaign_id', 'Site', 'weekly_avg_sales_amount', 'stddev_sales_amount', 'campaign_week_sales', 'percentage_uplift_vs_historical_avg']],
    how='left',
    left_on=['study_id', 'test_store'],
    right_on=['campaign_id', 'Site']
)

# Rename the columns to reflect they are for the test_store
merged_test = merged_test.rename(columns={
    'weekly_avg_sales_amount': 'weekly_avg_test',
    'stddev_sales_amount': 'stddev_test',
    'campaign_week_sales': 'campaign_sales_test',
    'percentage_uplift_vs_historical_avg': 'percentage_uplift_test'
})

# Drop the redundant 'Site' column
merged_test = merged_test.drop(columns=['Site'])

# Next, merge check_df onto the result for the control_store
merged_full = pd.merge(
    merged_test,
    check_df[['campaign_id', 'Site', 'weekly_avg_sales_amount', 'stddev_sales_amount', 'campaign_week_sales', 'percentage_uplift_vs_historical_avg']],
    how='left',
    left_on=['study_id', 'control_store'],
    right_on=['campaign_id', 'Site']
)

# Rename the columns to reflect they are for the control_store
merged_full = merged_full.rename(columns={
    'weekly_avg_sales_amount': 'weekly_avg_control',
    'stddev_sales_amount': 'stddev_control',
    'campaign_week_sales': 'campaign_sales_control',
    'percentage_uplift_vs_historical_avg': 'percentage_uplift_control'
})

# Drop the redundant 'Site' column
merged_full = merged_full.drop(columns=['Site', 'study_id'])
#merged_full['incremental'] = merged_full['percentage_uplift_test'] - merged_full['percentage_uplift_control']
merged_full['weekly_avg_test'] = merged_full['weekly_avg_test'].astype(float)
merged_full['weekly_avg_control'] = merged_full['weekly_avg_control'].astype(float)
merged_full['stddev_test'] = merged_full['stddev_test'].astype(float)
merged_full['stddev_control'] = merged_full['stddev_control'].astype(float)
merged_full['cv_test'] = merged_full['stddev_test'] / merged_full['weekly_avg_test']
merged_full['cv_control'] = merged_full['stddev_control'] / merged_full['weekly_avg_control']
merged_full['cv_test'] = merged_full['cv_test'].astype(float)
merged_full['cv_control'] = merged_full['cv_control'].astype(float)
merged_full['percentage_uplift_test'] = merged_full['percentage_uplift_test'].astype(float)
merged_full['percentage_uplift_control'] = merged_full['percentage_uplift_control'].astype(float)
merged_full['standardized_uplift_test'] = merged_full['percentage_uplift_test'] / merged_full['cv_test']
merged_full['standardized_uplift_control'] = merged_full['percentage_uplift_control'] / merged_full['cv_control']
merged_full['standardized_incremental_uplift'] = merged_full['standardized_uplift_test'] - merged_full['standardized_uplift_control']
merged_full['adjusted_incremental_effect'] = merged_full['standardized_incremental_uplift'] * merged_full['cv_test']
merged_full


,test_store,control_store,abs_perc_diff,campaign_id_x,weekly_avg_test,stddev_test,campaign_sales_test,percentage_uplift_test,campaign_id_y,weekly_avg_control,stddev_control,campaign_sales_control,percentage_uplift_control,cv_test,cv_control,standardized_uplift_test,standardized_uplift_control,standardized_incremental_uplift,adjusted_incremental_effect
0,2640,3175,0.000400,WOW20000860_1,13.717356,8.952424,156.950000000,10.441709,WOW20000860_1,13.720694,8.951020,177.330000000,11.924273,0.652635,0.652374,15.999314,18.278287,-2.278973,-1.487337
1,1198,2610,0.002398,WOW20000860_1,17.858250,12.944477,173.100000000,8.692999,WOW20000860_1,17.828608,12.954009,179.890000000,9.089964,0.724846,0.726586,11.992894,12.510521,-0.517626,-0.375199
2,3395,1470,0.002417,WOW20000860_1,15.513429,10.738371,92.700000000,4.975468,WOW20000860_1,15.548519,10.740087,185.880000000,10.954837,0.692198,0.690747,7.187922,15.859414,-8.671492,-6.002393
3,3210,3126,0.002912,WOW20000860_1,18.768028,11.289037,101.800000000,4.424118,WOW20000860_1,18.737738,11.303679,76.620000000,3.089074,0.601504,0.603257,7.355098,5.120657,2.234440,1.344024
4,3160,1167,0.003415,WOW20000860_1,22.246556,17.345718,101.840000000,3.577787,WOW20000860_1,22.255246,17.293416,71.260000000,2.201942,0.779704,0.777049,4.588651,2.833723,1.754928,1.368323
5,2670,2951,0.003636,WOW20000860_1,30.718632,22.167914,322.510000000,9.498840,WOW20000860_1,30.767826,22.122848,289.400000000,8.405929,0.721644,0.719025,13.162780,11.690727,1.472053,1.062298
6,1190,3368,0.003767,WOW20000860_1,15.089125,9.812585,103.270000000,5.844002,WOW20000860_1,15.143171,9.810640,121.930000000,7.051814,0.650308,0.647859,8.986508,10.884797,-1.898290,-1.234474
7,3321,1368,0.004450,WOW20000860_1,16.286400,10.953601,140.300000000,7.614550,WOW20000860_1,16.271207,10.992251,59.630000000,2.664756,0.672561,0.675565,11.321720,3.944487,7.377232,4.961640
8,5654,3053,0.004607,WOW20000860_1,19.032532,13.133237,174.050000000,8.144869,WOW20000860_1,19.074588,13.101759,94.030000000,3.929595,0.690042,0.686870,11.803447,5.721019,6.082428,4.197128
9,1147,1154,0.004911,WOW20000860_1,22.299028,14.810586,115.130000000,4.163005,WOW20000860_1,22.327303,14.864763,155.420000000,5.960984,0.664181,0.665766,6.267880,8.953570,-2.685690,-1.783784


In [52]:
merged_full['adjusted_incremental_effect'].mean()

-0.08521257765294996

In [91]:
check = client.query(
f"""
#SELECT ppu.* FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_historical_vs_campaign_price_per_unit ppu
#SELECT COUNT(DISTINCT Article) AS nskus FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_historical_vs_campaign_price_per_unit ppu
with step_one AS (
    SELECT 
        weeks_count, 
        Site,
        COUNT(DISTINCT Article) AS n_skus
    FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_baseline_statistics_avg_price_per_unit
    GROUP BY 1,2
),
step_two AS (
    SELECT 
        weeks_count, 
        n_skus, 
        COUNT(DISTINCT Site) AS n_stores 
    FROM step_one 
    GROUP BY 1,2
)
SELECT 
    weeks_count, 
    n_skus, 
    n_stores
FROM step_two 
ORDER BY weeks_count
#SELECT * FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_campaign_period_avg_price_per_unit ORDER BY Article
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,weeks_count,n_skus,n_stores
0,1,2,73
1,1,1,329
2,1,3,6
3,1,4,1
4,2,2,86
5,2,1,411
6,2,3,16
7,2,4,1
8,3,1,314
9,3,2,77


In [86]:
check = client.query(
f"""
#SELECT ppu.* FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_historical_vs_campaign_price_per_unit ppu
SELECT 
    ppu.* 
FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_historical_vs_campaign_price_per_unit ppu

#SELECT * FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_baseline_statistics_avg_price_per_unit WHERE campaign_id IS NULL OR campaign_id = ""
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,campaign_id,Site,Article,historical_n_weeks,historical_avg_ppu,campaign_avg_ppu
0,WOW20000860_1,2755,10044,1,6.180000000,None
1,WOW20000860_1,7222,10044,1,4.500000000,4.305000000
2,WOW20000860_1,2726,10044,2,4.885000000,None
3,WOW20000860_1,2340,10044,2,5.785000000,None
4,WOW20000860_1,1446,10044,2,5.525000000,None
...,...,...,...,...,...,...
9425,WOW20000860_1,1136,839570,12,5.537600000,4.071666667
9426,WOW20000860_1,5670,839570,12,5.326774194,4.191666667
9427,WOW20000860_1,2644,839570,12,5.282117647,4.140526316
9428,WOW20000860_1,4397,839570,12,5.326851852,4.095833333


In [74]:
check = client.query(
    f"""    
    SELECT * 
    FROM cart-dai-sandbox-nonprod-c3e7.{data_scientist}.regularization_process_baseline_statistics_with_campaign
    WHERE weeks_count = 12
    """
).result()
check_df = check.to_dataframe()
check_df

,campaign_id,Site,weeks_count,weekly_avg_sales_amount,stddev_sales_amount,coefficient_of_variation
0,WOW20000860_1,1280,12,105.636666667,31.701112,0.300096
1,WOW20000860_1,5003,12,270.825833333,81.870596,0.302300
2,WOW20000860_1,2636,12,237.556666667,87.468394,0.368200
3,WOW20000860_1,3195,12,154.145000000,65.869322,0.427321
4,WOW20000860_1,5601,12,63.806666667,48.857333,0.765709
...,...,...,...,...,...,...
958,WOW20000860_1,5639,12,240.428333333,82.016267,0.341126
959,WOW20000860_1,2643,12,252.875000000,51.086239,0.202022
960,WOW20000860_1,3223,12,63.803333333,40.374397,0.632794
961,WOW20000860_1,5691,12,38.070000000,27.068049,0.711007


In [ ]:
check = client.query(
"""
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

In [65]:
merged_df = pd.read_csv("~/Documents/Cartology/WOW20000860_1_merge_df.csv")
merged_df['transformed_campaign_week_uplift'] = merged_df['standardized_campaign_week_uplift'] * merged_df['coefficient_of_variation']
merged_df.sort_values('transformed_campaign_week_uplift')

,Unnamed: 0,total_sum_sales,store,test_or_control,campaign_id,stddev_sales_amount,weekly_avg_sales_amount,coefficient_of_variation,campaign_week_uplift,standardized_campaign_week_uplift,transformed_campaign_week_uplift
873649,873649,26.955946,3178,Control,WOW20000860_1,18.400137,32.631553,0.563876,-0.173930,-0.308455,-0.173930
320765,320765,12.556103,1413,Control,WOW20000860_1,7.830007,10.179714,0.769177,0.233444,0.303498,0.233444
320609,320609,12.633754,1413,Control,WOW20000860_1,7.830007,10.179714,0.769177,0.241072,0.313415,0.241072
320710,320710,12.652896,1413,Control,WOW20000860_1,7.830007,10.179714,0.769177,0.242952,0.315859,0.242952
325054,325054,12.664918,1413,Control,WOW20000860_1,7.830007,10.179714,0.769177,0.244133,0.317395,0.244133
...,...,...,...,...,...,...,...,...,...,...,...
38813,38813,763.834146,1104,Control,WOW20000860_1,15.164095,20.099286,0.754459,37.003049,49.045778,37.003049
34266,34266,765.345034,1104,Control,WOW20000860_1,15.164095,20.099286,0.754459,37.078221,49.145414,37.078221
33857,33857,773.914410,1104,Control,WOW20000860_1,15.164095,20.099286,0.754459,37.504573,49.710524,37.504573
37803,37803,774.913231,1104,Control,WOW20000860_1,15.164095,20.099286,0.754459,37.554267,49.776391,37.554267
